In [152]:
import pandas as pd

# Cleaning up PG & E data 

In [175]:
# manipulating electrical data
edf = pd.read_csv("./data/electrical.csv")                    # read the data
edf['DATE'] = edf['DATE'].astype('datetime64')                # set the date type
edf['COST'] = edf['COST'].str.replace('$','')                 # remove unnecesary symbols
edf['COST'] = edf['COST'].astype('float64')                   # set the right type
edf = edf.set_index('DATE').resample('D').sum()               # Get the usage and cost total per day


# manipulating gas data
tmp = pd.read_csv("./data/gas.csv").drop(columns=['TYPE','UNITS','NOTES'])
gdf = pd.DataFrame()
gdf['GAS_DATE'] = tmp['DATE'].astype('datetime64')
tmp['COST'] = tmp['COST'].str.replace('$','')
gdf['GAS_COST'] = tmp['COST'].astype('float64')
gdf['GAS_USAGE'] = tmp['USAGE']

gdf = gdf.set_index('GAS_DATE')


pge_data = pd.concat([edf, gdf], axis=1, join='inner')     # join dataframes

# renaming columns
pge_data.columns = ['electrical_usage','electrical_cost','gas_cost','gas_usage']
pge_data.index.name = 'DATE'

# adding a total_cost column
pge_data['total_cost'] = pge_data['electrical_cost'] + pge_data['gas_cost']


In [176]:
pge_data

,electrical_usage,electrical_cost,gas_cost,gas_usage,total_cost
DATE,,,,,
2014-07-30,0.48,0.00,0.00,0.00,0.00
2014-07-31,0.48,0.00,0.00,0.00,0.00
2014-08-01,0.48,0.00,0.00,0.00,0.00
2014-08-02,0.48,0.00,0.00,0.00,0.00
2014-08-03,0.48,0.00,0.00,0.00,0.00
...,...,...,...,...,...
2020-07-06,30.64,7.48,0.49,0.35,7.97
2020-07-07,55.99,13.64,0.49,0.35,14.13
2020-07-08,38.04,9.28,0.00,0.00,9.28


# Time to cleanup the weather data

In [180]:
wdf = pd.read_csv('data/weather.csv')
wdf.columns

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'AWND_ATTRIBUTES', 'DAPR', 'DAPR_ATTRIBUTES', 'MDPR', 'MDPR_ATTRIBUTES',
       'PGTM', 'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'SNOW',
       'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'TOBS',
       'TOBS_ATTRIBUTES', 'WDF2', 'WDF2_ATTRIBUTES', 'WDF5', 'WDF5_ATTRIBUTES',
       'WSF2', 'WSF2_ATTRIBUTES', 'WSF5', 'WSF5_ATTRIBUTES', 'WT01',
       'WT01_ATTRIBUTES', 'WT02', 'WT02_ATTRIBUTES', 'WT03', 'WT03_ATTRIBUTES',
       'WT08', 'WT08_ATTRIBUTES'],
      dtype='object')

In [183]:
wdf = wdf.loc[:,['DATE','PRCP','TMAX','TMIN']]
wdf.set_index('DATE')

,PRCP,TMAX,TMIN
DATE,,,
2014-07-01,0.0,102.0,57.0
2014-07-02,0.0,100.0,63.0
2014-07-03,0.0,98.0,63.0
2014-07-04,0.0,98.0,63.0
2014-07-05,0.0,102.0,63.0
...,...,...,...
2020-07-03,0.0,94.0,60.0
2020-07-04,0.0,97.0,58.0
2020-07-05,0.0,99.0,62.0


In [188]:
pd.concat([pge_data,wdf], axis=1).head()     # join dataframes

,electrical_usage,electrical_cost,gas_cost,gas_usage,total_cost,DATE,PRCP,TMAX,TMIN
1970-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01 00:00:00.000000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01 00:00:00.000000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01 00:00:00.000000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01 00:00:00.000000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Saving the data to file
pge_data.to_csv("data/pge_data.csv")                       # save data